# Assignment 3: Predicting Airbnb Prices

## Pre-processing steps:

In [10]:
# Import gurobi and numpy
from gurobipy import *
import pandas as pd
import numpy as np
from numpy import genfromtxt
import csv

# Load the training and test datasets
df_train = pd.read_csv('AirbnbTrain.csv')
df_test = pd.read_csv('AirbnbTest.csv')

x_train = df_train.loc[:, df_train.columns!='price'] # Independent variable values for training set
y_train = df_train.loc[:, df_train.columns=='price'] # Predictor variable values for training set

x_test = df_test.loc[:, df_test.columns!='price'] # Independent variable values for training set
y_test = df_test.loc[:, df_test.columns=='price'] # Predictor variable values for training set

Display contents of training and test datasets below:

In [4]:
df_train.head()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.103701,-118.332241,1,13,2.0,3,2,150,2,1,100,1,350
1,34.099484,-118.331645,1,8,2.0,2,4,150,1,11,96,1,190
2,34.104321,-118.329662,1,4,1.0,0,1,55,1,1,80,0,85
3,34.101028,-118.317848,0,2,1.0,1,1,20,1,8,98,0,75
4,34.098292,-118.324980,1,2,1.0,1,1,20,1,11,96,0,130


In [5]:
df_test.head()

,latitude,longitude,Entire home,accommodates,bathrooms,bedrooms,beds,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,instant_bookable,price
0,34.100604,-118.341787,0,2,1.0,1,1,40,1,261,96,1,100
1,34.100607,-118.350583,1,8,2.0,2,2,100,2,10,98,0,300
2,34.100610,-118.347617,1,2,1.0,1,1,80,2,1,100,1,125
3,34.100611,-118.342180,1,3,1.0,0,2,55,2,54,97,1,169
4,34.100618,-118.342791,1,4,1.0,1,1,70,2,233,92,1,119


## Question 1 - Model 1

Formulating the least absolute deviations regression problem on training data as a linear program below:

In [70]:
# Define model and parameters. 
mod_1 = Model()

n_i = 1700 # Rows of observed data
n_j = 12 # Total number of independent variables

x = x_train.to_numpy() # independent variable values from training set
y = y_train.to_numpy() # predictor variable values from training set
y = y.flatten() # turn predictor variable into a 1-d array

# Define decision variables.
Z = mod_1.addVars(n_i)
B = mod_1.addVars(n_j)

# Absolute value constraint 1
abs_con_1 = {}
for i in range(n_i):
    abs_con_1[i] = mod_1.addConstr(Z[i] >= y[i] - sum(x[i, j] * B[j] for j in range(n_j)))

# Absolute value constraint 2
abs_con_2 = {}
for i in range(n_i):
    abs_con_2[i] = mod_1.addConstr(Z[i] >= sum(x[i, j] * B[j] for j in range(n_j)) - y[i])
    
# Create the objective function for SAE, and set it to be minimized.
mod_1.setObjective((1 / n_i) * sum(Z[i] for i in range(n_i)), GRB.MINIMIZE)

mod_1.update()

mod_1.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3400 rows, 1712 columns and 41372 nonzeros
Model fingerprint: 0x49c47e75
Coefficient statistics:
  Matrix range     [5e-01, 5e+02]
  Objective range  [6e-04, 6e-04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve time: 0.01s
Presolved: 3400 rows, 1712 columns, 41372 nonzeros

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 12
 AA' NZ     : 2.995e+04
 Factor NZ  : 3.260e+04 (roughly 2 MB of memory)
 Factor Ops : 4.141e+05 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.13176381e+03  0.00000000e+00  1.36e+03 0.00e+00  2.72e+01     0s
   1   1.42686270e+03  1.34119445e

Finding the prediction error of the model on the test set below:

In [77]:
# Define model and parameters. 
mod_1_test = Model()

n_i_test = 699 # Rows of observed data
n_j = 12 # Total number of independent variables

x_t = x_test.to_numpy() # independent variable values from test set
y_t = y_test.to_numpy() # predictor variable values from test set
y_t = y_t.flatten() # turn predictor variable into a 1-d array

B_train_mod_1 = [B[j].x for j in range(n_j)] # Get values for each coefficient in training set

# Define decision variables.
Z = mod_1_test.addVars(n_i)

# Absolute value constraint 1
abs_con_1_test = {}
for i in range(n_i_test):
    abs_con_1_test[i] = mod_1_test.addConstr(Z[i] >= y_t[i] - sum(x_t[i, j] * B_train_mod_1[j] for j in range(n_j)))

# Absolute value constraint 2
abs_con_2_test = {}
for i in range(n_i_test):
    abs_con_2_test[i] = mod_1_test.addConstr(Z[i] >= sum(x_t[i, j] * B_train_mod_1[j] for j in range(n_j)) - y_t[i])
    
# Create the objective function for SAE, and set it to be minimized.
mod_1_test.setObjective((1 / n_i_test) * sum(Z[i] for i in range(n_i)), GRB.MINIMIZE)

mod_1_test.update()

mod_1_test.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1398 rows, 1700 columns and 1398 nonzeros
Model fingerprint: 0x456a898a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-14, 9e+02]
Presolve removed 1398 rows and 1700 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5604535e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.560453503e+01


The prediction error of our model on the test set is **35.60** in dollars per night.

## Question 2 - Model 2

### Part A

In [87]:
# Define model and parameters. 
mod_2 = Model()

n_i = 1700 # Rows of observed data
n_j = 12 # Total number of independent variables
M = 100000 # Cut off for our feasible solutions

x = x_train.to_numpy() # independent variable values from training set
y = y_train.to_numpy() # predictor variable values from training set
y = y.flatten() # turn predictor variable into a 1-d array

# Define decision variables.
Z = mod_2.addVars(n_i)
B = mod_2.addVars(n_j)
V = mod_2.addVars(n_j, vtype = GRB.BINARY)

# Absolute value constraint 1
abs_con_1 = {}
for i in range(n_i):
    abs_con_1[i] = mod_2.addConstr(Z[i] >= y[i] - sum(x[i, j] * B[j] for j in range(n_j)))

# Absolute value constraint 2
abs_con_2 = {}
for i in range(n_i):
    abs_con_2[i] = mod_2.addConstr(Z[i] >= sum(x[i, j] * B[j] for j in range(n_j)) - y[i])
    
# Binary value and inclusion of each independent variable constraint
V_con_1 = {}
for j in range(n_j):
    V_con_1[j] = mod_2.addConstr(B[j] <= M * V[j])
    
V_con_2 = {}
for j in range(n_j):
    V_con_2[j] = mod_2.addConstr(B[j] >= -M * V[j])

# Three non-zero coefficients at most constraint
lasso_con = mod_2.addConstr(sum(V[j] for j in range(n_j)) <= 3)
    
# Create the objective function for SAE, and set it to be minimized.
mod_2.setObjective((1 / n_i) * sum(Z[i] for i in range(n_i)), GRB.MINIMIZE)

mod_2.update()

mod_2.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3425 rows, 1724 columns and 41432 nonzeros
Model fingerprint: 0x06057cd5
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+05]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 840 rows and 414 columns
Presolve time: 0.03s
Presolved: 2585 rows, 1310 columns, 31274 nonzeros
Variable types: 1298 continuous, 12 integer (12 binary)

Root relaxation: objective 3.642625e+01, 1385 iterations, 0.14 seconds (0.36 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.42625    0    5  144.96824   36.42625  74.9%     -    0s
H    0     0         

In [89]:
# Find coefficients for the three variables below:
B_train_mod_2 = [B[j].x for j in range(n_j)]
print(B_train_mod_2)

[0.0, 0.0, 52.0, 14.0, 0.0, 32.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Here are the names and coefficients of the three variables selected by the optimization model: <br>
- Entire home: **52.0**
- Accomodates: **14.0**
- Bedrooms: **32.0**

### Part B

In [91]:
# Define model and parameters. 
mod_2_test = Model()

n_i_test = 699 # Rows of observed data
n_j = 12 # Total number of independent variables

x_t = x_test.to_numpy() # independent variable values from test set
y_t = y_test.to_numpy() # predictor variable values from test set
y_t = y_t.flatten() # turn predictor variable into a 1-d array

B_train_mod_2 = [B[j].x for j in range(n_j)] # Get values for each coefficient in training set

# Define decision variables.
Z = mod_2_test.addVars(n_i)

# Absolute value constraint 1
abs_con_1_test = {}
for i in range(n_i_test):
    abs_con_1_test[i] = mod_2_test.addConstr(Z[i] >= y_t[i] - sum(x_t[i, j] * B_train_mod_2[j] for j in range(n_j)))

# Absolute value constraint 2
abs_con_2_test = {}
for i in range(n_i_test):
    abs_con_2_test[i] = mod_2_test.addConstr(Z[i] >= sum(x_t[i, j] * B_train_mod_2[j] for j in range(n_j)) - y_t[i])
    
# Create the objective function for SAE, and set it to be minimized.
mod_2_test.setObjective((1 / n_i_test) * sum(Z[i] for i in range(n_i)), GRB.MINIMIZE)

mod_2_test.update()

mod_2_test.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1398 rows, 1700 columns and 1398 nonzeros
Model fingerprint: 0x1e3a7b6a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 9e+02]
Presolve removed 1398 rows and 1700 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7736767e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.773676681e+01


The prediction error of model 2 on the test set is **37.74** in dollars per night.

## Question 3 - Model 3

### Part A

In [104]:
# Define model and parameters. 
mod_3 = Model()

n_i = 1700 # Rows of observed data
n_j = 12 # Total number of independent variables
M = 100000 # Cut off for our feasible solutions

x = x_train.to_numpy() # independent variable values from training set
y = y_train.to_numpy() # predictor variable values from training set
y = y.flatten() # turn predictor variable into a 1-d array

# Define decision variables.
Z = mod_3.addVars(n_i)
B = mod_3.addVars(n_j)
V = mod_3.addVars(n_j, vtype = GRB.BINARY)

# Absolute value constraint 1
abs_con_1 = {}
for i in range(n_i):
    abs_con_1[i] = mod_3.addConstr(Z[i] >= y[i] - sum(x[i, j] * B[j] for j in range(n_j)))

# Absolute value constraint 2
abs_con_2 = {}
for i in range(n_i):
    abs_con_2[i] = mod_3.addConstr(Z[i] >= sum(x[i, j] * B[j] for j in range(n_j)) - y[i])
    
# Binary value and inclusion of each independent variable constraint
V_con_1 = {}
for j in range(n_j):
    V_con_1[j] = mod_3.addConstr(B[j] <= M * V[j])
    
V_con_2 = {}
for j in range(n_j):
    V_con_2[j] = mod_3.addConstr(B[j] >= -M * V[j])

# Choosing "number of beds" to be one of our variables constraint
beds_con = mod_3.addConstr(V[6] == 1)

# Having exactly three non-zero coefficients constraint
lasso_con = mod_3.addConstr(sum(V[j] for j in range(n_j)) == 3)
    
# Create the objective function for SAE, and set it to be minimized.
mod_3.setObjective((1 / n_i) * sum(Z[i] for i in range(n_i)), GRB.MINIMIZE)

mod_3.update()

mod_3.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 3426 rows, 1724 columns and 41433 nonzeros
Model fingerprint: 0x86da9d44
Variable types: 1712 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+05]
  Objective range  [6e-04, 6e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 144.9682353
Presolve removed 842 rows and 415 columns
Presolve time: 0.03s
Presolved: 2584 rows, 1309 columns, 31271 nonzeros
Variable types: 1298 continuous, 11 integer (11 binary)

Root relaxation: objective 3.642625e+01, 1382 iterations, 0.14 seconds (0.35 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.42625    0    6  144.96824   36.42625  74.9%     -    0s
H    0     0         

In [105]:
# Find coefficients for the three variables below:
B_train_mod_3 = [B[j].x for j in range(n_j)]
print(B_train_mod_3)

[0.0, 0.0, 67.875, 0.0, 0.0, 47.375, 12.125, 0.0, 0.0, 0.0, 0.0, 0.0]


Here are the names and coefficients of the two other variables selected by the optimization model: <br>
- Entire home: **67.875**
- Bedrooms: **47.375**

### Part B

The variable in Model 2 that is no longer in Model 3 is the "accommodates" variable. This is most likely due to us forcing "number of beds" to be one of our independent variables, and "accommodates" was excluded because the two are highly correlated. Because they are highly correlated, having both variables in our model wouldn't be as helpful to making a prediction on price so the model decided it was best to choose other variables that would be more useful in making the prediction on price and not as correlated with "number of beds".

### Part C

In [106]:
# Define model and parameters. 
mod_3_test = Model()

n_i_test = 699 # Rows of observed data
n_j = 12 # Total number of independent variables

x_t = x_test.to_numpy() # independent variable values from test set
y_t = y_test.to_numpy() # predictor variable values from test set
y_t = y_t.flatten() # turn predictor variable into a 1-d array

B_train_mod_3 = [B[j].x for j in range(n_j)] # Get values for each coefficient in training set

# Define decision variables.
Z = mod_3_test.addVars(n_i)

# Absolute value constraint 1
abs_con_1_test = {}
for i in range(n_i_test):
    abs_con_1_test[i] = mod_3_test.addConstr(Z[i] >= y_t[i] - sum(x_t[i, j] * B_train_mod_3[j] for j in range(n_j)))

# Absolute value constraint 2
abs_con_2_test = {}
for i in range(n_i_test):
    abs_con_2_test[i] = mod_3_test.addConstr(Z[i] >= sum(x_t[i, j] * B_train_mod_3[j] for j in range(n_j)) - y_t[i])
    
# Create the objective function for SAE, and set it to be minimized.
mod_3_test.setObjective((1 / n_i_test) * sum(Z[i] for i in range(n_i)), GRB.MINIMIZE)

mod_3_test.update()

mod_3_test.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1398 rows, 1700 columns and 1398 nonzeros
Model fingerprint: 0x4307ef6f
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 9e+02]
Presolve removed 1398 rows and 1700 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8599607e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.859960658e+01


The prediction error of model 3 on the test set is **38.60** in dollars per night.